In [1]:
#Imports
import numpy as np
from midiutil import MIDIFile
import matplotlib.pyplot as plt
import ternary
from ternary.helpers import simplex_iterator
import graphviz
import anytree as at
from anytree import Node, RenderTree
from anytree.exporter import DotExporter

# helpers
import sys
sys.path.insert(0, 'helpers/')
from tree_parser import TreeParser
from midi import *

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# Templates level 1
templates_1 = set()
templates_1.add("[p1]")
templates_1.add("[p2]")
templates_1.add("[p3]")

# Templates level 2
templates_2 = set()
templates_2.add("[p2]")
templates_2.add("[p3]")

for elem1 in templates_1:
    for elem2 in templates_1:
        templates_2.add((elem1, elem2))

# Templates level 3
templates_3 = set()
templates_3.add("[p2]")
templates_3.add("[p3]")

for elem1 in templates_2:
    for elem2 in templates_2:
        templates_3.add((elem1, elem2))
        
# Templates level 4
templates_4 = set()
templates_4.add("[p2]")
templates_4.add("[p3]")

for elem1 in templates_3:
    for elem2 in templates_3:
        templates_4.add((elem1, elem2))
        
# Templates level 5
templates_5 = set()
templates_5.add("[p2]")
templates_5.add("[p3]")

for elem1 in templates_4:
    for elem2 in templates_4:
        templates_5.add((elem1, elem2))
    

templates_5

In [11]:
type(("[p1]"))

str